# Web Scraping Car Prices at Bilweb.se

### OBS! Only for educational purposes

# 1) Import Packages 

In [26]:
import os
os.chdir('..') # Running this to in order to be able to import local files.
import requests
from bs4 import BeautifulSoup
import bs4
import json
import re as re
import pandas as pd
import seaborn as sns
from collections import Counter
from datetime import datetime,date,timedelta
import numpy as np
import sqlite3
import pickle
import utils.db_queries # Local File
import utils.car_prices_functions # Local File

# 2) Extracting, Manipulating and Inserting the data

In [3]:
# Here we are importing the mapping between the unique code for a given car brand on Bilweb. Keep in mind that the 
# mapping can change on their end. This mapping was collected from the response and stored as a dictionary for future use.
with open('car_brand_dict.pkl', 'rb') as handle:
    car_brand_dict = pickle.load(handle)

In [88]:
# Pulling a given card brand for the Stockholms Region. Automatic cars only. 
car_brand_code = car_brand_dict['Volvo']
x = requests.get(f"https://bilweb.se/sok?offset=0&limit=1000&order_by=timestamp&order=desc&type=1&brand={car_brand_code}&have_mrf_association=&query=&price_min=0&price_max=500000&year_min=&year_max=&property_mileage_min=&property_mileage_max=&fuel%5B%5D=1&fuel%5B%5D=4&fuel%5B%5D=5&gear_box%5B%5D=2&region=12")
html=x.content
soup = BeautifulSoup(html)

In [89]:
# Setting up a connection to a local SQLite3 Database
sql_connection = dq.create_connection('car_prices.db')
cursor = sql_connection.cursor()

In [90]:
# Create the dataset from the data points that are extracted from the html.
car_dataset, car_dataset_copy = cpf.create_car_dataset(soup) 

#Insert dataset into the local SQLite3 Database
dq.insert_into_db(car_dataset)

# 3) Extracting data from the database

In [91]:
sql = "SELECT * FROM CAR_DATASET"
cursor.execute(sql)
rows = cursor.fetchall()

In [92]:
full_dataset = pd.DataFrame(rows)
full_dataset.columns = ['PRICE','TIMESTAMP','CAR_STRING','YEAR','MILES','CAR_MODEL']

In [93]:
full_dataset.query("CAR_STRING.str.contains('Tesla') & PRICE < 400000").head(50).sort_values(by = 'PRICE')

,PRICE,TIMESTAMP,CAR_STRING,YEAR,MILES,CAR_MODEL
482,0,02-01,Tesla Model 3 Long Range Dual Motor AWD 2019,2019,6907,Tesla
488,0,02-01,Tesla Model 3 Long Range Dual Motor AWD 2019,2019,5126,Tesla
487,280000,02-01,Tesla Model S 60 2013,2013,24805,Tesla
472,332000,02-03,Tesla Model S P85D 2015,2015,17885,Tesla
528,371900,01-19,Tesla Model 3 Standard Range Plus Pano Autopil...,2020,8010,Tesla
533,377900,01-18,Tesla Model S 85 Pano Backkamera Svensksåld 2014,2014,17528,Tesla
491,378000,01-31,Tesla Model 3 Performance AWD 2019,2019,9924,Tesla
516,384800,01-23,Tesla Model 3 Standard Range Plus Svensksåld 2019,2019,5297,Tesla
693,385000,11-09,Tesla Model 3 Standard Range Plus RWD Pano Nav...,2020,6910,Tesla
580,388000,12-28,Tesla Model 3 Standard Range Plus RWD Drag Nav...,2019,5935,Tesla
